In [1]:
import os
import time
import logging
import pandas as pd
from sqlalchemy import create_engine
import psutil
from datetime import datetime
#data = r'C:\Downloads\Project_Alpha\data'

In [2]:
# Create logs folder if it doesn't exist
log_dir = "logs"
os.makedirs(log_dir, exist_ok=True)

# Optional: dynamic log file name with timestamp


log_file = os.path.join("logs", f"ingestion_log_{datetime.now().strftime('%Y%m%d_%H%M%S')}.txt")



In [3]:

# Set up logging
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s',
    handlers=[
        logging.StreamHandler(),
        logging.FileHandler(log_file, mode='a', encoding='utf-8')  # Save to logs/ folder
    ]
)

In [4]:
# === DB and Data path === #
data = r'C:\Downloads\Project_Alpha\data'
engine = create_engine('sqlite:///inventory.db')

In [5]:
# === Memory usage helper === #
def get_memory_usage_mb():
    process = psutil.Process(os.getpid())
    return round(process.memory_info().rss / 1024 / 1024, 2)  # in MB

In [6]:
# === Ingest CSV with chunking and logging === #
def ingest_large_csv(file_path, table_name, engine, chunksize=10000):
    logging.info(f"📥 Starting ingestion: {file_path}")
    start_time = time.time()
    
    try:
        for i, chunk in enumerate(pd.read_csv(file_path, chunksize=chunksize)):
            mode = 'replace' if i == 0 else 'append'
            chunk.to_sql(table_name, con=engine, if_exists=mode, index=False)
            mem_usage = get_memory_usage_mb()
            logging.info(f"✅ Chunk {i + 1} into '{table_name}' → Rows: {chunk.shape[0]}, Mem: {mem_usage} MB")
        
        elapsed = round(time.time() - start_time, 2)
        logging.info(f"✅ Completed ingestion for '{table_name}' in {elapsed} sec\n")

    except Exception as e:
        logging.error(f"❌ Error during ingestion of '{file_path}': {e}")

In [7]:
# === Loop over CSV files === #
if __name__ == '__main__':
    logging.info(f"🔍 Scanning directory: {data}\n")
    
    for file in os.listdir(data):
        if file.endswith('.csv'):
            file_path = os.path.join(data, file)
            table_name = file[:-4]  # Remove .csv
            logging.info(f"📄 Processing file: {file}")
            ingest_large_csv(file_path, table_name, engine)

2025-08-12 00:47:57,712 - INFO - 🔍 Scanning directory: C:\Downloads\Project_Alpha\data

2025-08-12 00:47:57,714 - INFO - 📄 Processing file: begin_inventory.csv
2025-08-12 00:47:57,716 - INFO - 📥 Starting ingestion: C:\Downloads\Project_Alpha\data\begin_inventory.csv
2025-08-12 00:47:58,126 - INFO - ✅ Chunk 1 into 'begin_inventory' → Rows: 10000, Mem: 134.4 MB
2025-08-12 00:47:58,209 - INFO - ✅ Chunk 2 into 'begin_inventory' → Rows: 10000, Mem: 138.21 MB
2025-08-12 00:47:58,289 - INFO - ✅ Chunk 3 into 'begin_inventory' → Rows: 10000, Mem: 138.03 MB
2025-08-12 00:47:58,371 - INFO - ✅ Chunk 4 into 'begin_inventory' → Rows: 10000, Mem: 139.02 MB
2025-08-12 00:47:58,458 - INFO - ✅ Chunk 5 into 'begin_inventory' → Rows: 10000, Mem: 138.18 MB
2025-08-12 00:47:58,551 - INFO - ✅ Chunk 6 into 'begin_inventory' → Rows: 10000, Mem: 137.89 MB
2025-08-12 00:47:58,657 - INFO - ✅ Chunk 7 into 'begin_inventory' → Rows: 10000, Mem: 137.77 MB
2025-08-12 00:47:58,795 - INFO - ✅ Chunk 8 into 'begin_invento